# Setup

## Imports and drive setup

In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.6 MB 4.2 MB/s 
     |████████████████████████████████| 133 kB 28.4 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 170 kB 48.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [ ]:
# Imports
import os

# import os
import wandb
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, sampler
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import random

# Random seed fixed to reproduce results.
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
np.random.seed(0)

# Use GPU if available
GPU = True 
if GPU:
    device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(f'Using {device}')

  

Using cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Dataset class definition and Loading Data

### Dataset - Heatmap

In [ ]:
 # Dataset class
class MmwaveDataset(Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform
        self.class2index = {"knife":0, "spoon":1, "nothing":2}#, "nothing":3}#{"emtpy":0, "human":1, "pan":2, "bike":3}#{"cat":0, "dog":1}

    def gen_label(self,lab):
      label = [0,0,0]
      if "shovel" in lab:
        label[0] = 1
      if "rake" in lab:
        label[1] = 1
      if "human" in lab:
        label[2] = 1
      return torch.from_numpy(np.array(label))

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df["filename"][index]#self.df[index, "filename"]
        label = self.gen_label(self.df[" Label"][index]) #self.class2index[self.df[" Label"][index]]#self.df[index, "Label"]]
        heatmap = torch.from_numpy(np.loadtxt(os.path.join(self.images_folder, filename), delimiter=',')).view([1, 32, 32])#21::]), delimiter=',')).view([1, 64, 63])#pd.read_csv(os.path.join(self.images_folder, filename)))
        mean = heatmap.mean()
        heatmap=heatmap-mean
        if self.transform:
            heatmap = self.transform(heatmap)
        return heatmap, label

##### First get the mean and std to normalize (standardization) properly:

# mmwaveDataset = CustomDataset('/content/drive/MyDrive/mmWaveData/Dataset_ROI_Knife_Spoon_ROI/mmwaveRA.csv','/content/drive/MyDrive/mmWaveData')#CustomDataset('/content/drive/MyDrive/mmWaveData/Dataset_torch/mmwaveRA.csv','/content/drive/MyDrive/mmWaveData')
# loader = torch.utils.data.DataLoader(mmwaveDataset, batch_size=len(mmwaveDataset))#=batch_size*10

# print("loaded")

# data = next(iter(loader))

# print("next_iter")
# mean, std = data[0].mean(), data[0].std()
# print(mean)
# print(std)

# #ran once and kept:

mean = 677.4736
std = 2323.8167

# Define Gaussian Noise augmenation class
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

# Transforms for Data Augmentation

transform1_heatmap = transforms.Compose([
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    AddGaussianNoise(0., 1.)
])
transform2_heatmap = transforms.Compose([
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5)
])


### Dataset Heatmap + Voxel

In [ ]:
# Define Dataset class

# class MmwaveDataset_with_voxel(Dataset):
#     def __init__(self, csv_path, images_folder, transform_heatmap = None,transform_voxel = None):
#         self.df = pd.read_csv(csv_path)
#         self.images_folder = images_folder
#         self.transform_heatmap = transform_heatmap
#         self.transform_voxel = transform_voxel
#         self.class2index = {"knife":0, "spoon":1, "hand":2}#, "nothing":3}#{"emtpy":0, "human":1, "pan":2, "bike":3}#{"cat":0, "dog":1}

#     def __len__(self):
#         return len(self.df)
#     def __getitem__(self, index):
#         filename = self.df["filename"][index]#self.df[index, "filename"]
#         label = self.class2index[self.df[" Label"][index]]#self.df[index, "Label"]]

#         heatmap = torch.from_numpy(np.loadtxt(os.path.join(self.images_folder, filename)+"_heatmap.csv", delimiter=',')).view([1, 32, 32])#21::]), delimiter=',')).view([1, 64, 63])#pd.read_csv(os.path.join(self.images_folder, filename)))
#         voxel = torch.from_numpy(np.loadtxt(os.path.join(self.images_folder, filename)+"_voxel.csv", delimiter=',')).view([1, 26, 26, 26])
        
#         ####### Todo : keep like this?
#         # mean = heatmap.mean()
#         # heatmap=heatmap-mean

#         if self.transform_heatmap:
#             heatmap = self.transform_heatmap(heatmap)
#         if self.transform_voxel:
#             voxel = self.transform_voxel(voxel)
        
#         return heatmap, voxel, label

# Define Dataset class

class MmwaveDataset_with_voxel(Dataset):
    def __init__(self, csv_path, images_folder, transform_heatmap = None,transform_voxel = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform_heatmap = transform_heatmap
        self.transform_voxel = transform_voxel
        self.class2index = {"knife":1, "spoon":0}#, "nothing":3}#{"emtpy":0, "human":1, "pan":2, "bike":3}#{"cat":0, "dog":1}

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df["filename"][index]#self.df[index, "filename"]
        label = self.class2index[self.df[" Label"][index]]#self.df[index, "Label"]]

        heatmap = torch.from_numpy(np.loadtxt(os.path.join(self.images_folder, filename)+"_heatmap.csv", delimiter=',')).view([1, 32, 32])#21::]), delimiter=',')).view([1, 64, 63])#pd.read_csv(os.path.join(self.images_folder, filename)))
        voxel = torch.from_numpy(np.loadtxt(os.path.join(self.images_folder, filename)+"_voxel.csv", delimiter=',')).view([1, 26, 26, 26])
        
        ####### Todo : keep like this?
        # mean = heatmap.mean()
        # heatmap=heatmap-mean

        if self.transform_heatmap:
            heatmap = self.transform_heatmap(heatmap)
        if self.transform_voxel:
            voxel = self.transform_voxel(voxel)
        
        return heatmap, voxel, label


# Define some 3d augmenation classes

class RandomFlip_Voxel(object):
    def __init__(self, p=0.5,dim=1):
        self.p = p
        self.dim = dim # Note: flipping alond dim 0 will not result in a change, use with 1,2 or 3
        
    def __call__(self, tensor):
        if random.random() > self.p:
          tensor = torch.flip(tensor, [self.dim])
        return tensor 
    
    def __repr__(self):
        return self.__class__.__name__ + '(p={0},dim={1})'.format(self.p,self.dim)

class Normalize_Voxel(object):
    def __init__(self, mean=0,std=1):
        self.mean = mean
        self.std = std # Note: flipping alond dim 0 will not result in a change, use with 1,2 or 3
        
    def __call__(self, tensor):
        return (tensor - self.mean)/self.std
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0},std={1})'.format(self.mean,self.std)


transform1_voxel_1 = transforms.Compose([
    RandomFlip_Voxel(p=0.5, dim=3)
])
transform1_voxel_2 = transforms.Compose([
    RandomFlip_Voxel(p=0.5, dim=1),
    RandomFlip_Voxel(p=0.5, dim=3)
])
transform1_voxel_3 = transforms.Compose([
    RandomFlip_Voxel(p=0.5, dim=1),
    RandomFlip_Voxel(p=0.5, dim=2),
    RandomFlip_Voxel(p=0.5, dim=3)
])

 

### DataLoading function

In [ ]:
       
# # Function to get train and val loaders given params

# def get_data_loaders_from(path_train,path_val,batch_size=32,transforms_heatmap=None,transforms_voxel=None, datatype="heatmap"):
#   if datatype == "heatmap" :
#     mmwaveDataset = MmwaveDataset(path_train,path_train[:-13], transform=transforms_heatmap)#'/content/drive/MyDrive/mmWaveData', transform=transforms)
#     mmwaveDataset_val = MmwaveDataset(path_val,path_val[:-13])#'/content/drive/MyDrive/mmWaveData', transform=transforms)
#   elif datatype == "heatmap_voxel" :
#     mmwaveDataset = MmwaveDataset_with_voxel(path_train,path_train[:-13], transform_heatmap=transforms_heatmap,transform_voxel=transforms_voxel)
#     mmwaveDataset_val = MmwaveDataset_with_voxel(path_val,path_val[:-13])
  
#   # For shuffling :
#   size_train,size_val = len(mmwaveDataset),len(mmwaveDataset_val)
#   idxs_train,idxs_val = list(range(size_train)),list(range(size_val))
#   np.random.shuffle(idxs_train)
#   np.random.shuffle(idxs_val) 
#   train_sampler,val_sampler = SubsetRandomSampler(idxs_train),SubsetRandomSampler(idxs_val)
  
#   # Train_loader
#   train_loader = torch.utils.data.DataLoader(mmwaveDataset, batch_size=batch_size, 
#                                            sampler=train_sampler)
#   # Validation loader
#   validation_loader = torch.utils.data.DataLoader(mmwaveDataset_val, batch_size=batch_size,
#                                             sampler=val_sampler)

#   return train_loader, validation_loader

       
# Function to get train and val loaders given params

def get_data_loaders_from(path_train,path_val,batch_size=32,transforms_heatmap=None,transforms_voxel=None, datatype="heatmap"):
  if datatype == "heatmap" :
    mmwaveDataset = MmwaveDataset(path_train,path_train[:-13], transform=transforms_heatmap)#'/content/drive/MyDrive/mmWaveData', transform=transforms)
    mmwaveDataset_val = MmwaveDataset(path_val,path_val[:-13])#'/content/drive/MyDrive/mmWaveData', transform=transforms)
  elif datatype == "heatmap_voxel" :
    mmwaveDataset = MmwaveDataset_with_voxel(path_train,path_train[:-13], transform_heatmap=transforms_heatmap,transform_voxel=transforms_voxel)
    mmwaveDataset_val = MmwaveDataset_with_voxel(path_val,path_val[:-13], transform_heatmap= transforms.Normalize(mean=826.7405,std=2492.3940),transform_voxel=Normalize_Voxel(mean=12.3799,std=260.7471))#transforms.Normalize(mean=819.3264,std=2452.4735),transform_voxel=Normalize_Voxel(mean=4.0342,std=159.1961))
  
  # For shuffling :
  size_train,size_val = len(mmwaveDataset),len(mmwaveDataset_val)
  idxs_train,idxs_val = list(range(size_train)),list(range(size_val))
  np.random.shuffle(idxs_train)
  np.random.shuffle(idxs_val) 
  train_sampler,val_sampler = SubsetRandomSampler(idxs_train),SubsetRandomSampler(idxs_val)
  
  # Train_loader
  train_loader = torch.utils.data.DataLoader(mmwaveDataset, batch_size=batch_size, 
                                           sampler=train_sampler)
  # Validation loader
  validation_loader = torch.utils.data.DataLoader(mmwaveDataset_val, batch_size=batch_size,
                                            sampler=val_sampler)

  return train_loader, validation_loader

## WandB Init

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Networks



### For input 16x16



In [ ]:
# TODO
# Used for all Resnets:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU()]
    if pool: 
      layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

# Networks :

class model_CNN1_16(nn.Module):
    def __init__(self):
        super(model_CNN1_16, self).__init__()
    
        self.features = nn.Sequential(          
          # In : (1,16,16)
          nn.Conv2d(1, 16, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(),

          # In : (16,8,8)
          nn.Conv2d(16, 8,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(8),
          nn.ReLU(), 
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,8*4*4)
          nn.Linear(8*4*4, 3)
      )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN2_16(nn.Module):
    def __init__(self):
        super(model_CNN2_16, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,16,16)
          nn.Conv2d(1, 32, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),

          # In : (32,8,8)
          nn.Conv2d(32, 16,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(), 
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,16*4*4)
          nn.Linear(16*4*4, 128),
          nn.ReLU(),
          nn.Linear(128, 3)
          )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN3_16(nn.Module):
    def __init__(self):
        super(model_CNN3_16, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,16,16)
          nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (8,14,14)
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (16,6,6)
          nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,4,4)
          nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,8*6*6)
          nn.Linear(16*2*2, 3)
          )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)


class model_VGGNet_16(nn.Module):
    def __init__(self):
        super(model_VGGNet_16, self).__init__()
        self.features = nn.Sequential(
             # In : (1,16,16)
            nn.Conv2d(1, 16, 3, padding=1),  # Conv3
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),  # Conv4
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,8,8)
            nn.Conv2d(16, 32, 3, padding=1),  # Conv5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # Conv6
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,4,4)
            nn.Conv2d(32, 64, 3, padding=1),  # Conv7
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool4

            # In : (64,2,2)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv9
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv10
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(2 * 2 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return out


class model_ResNet_16(nn.Module):
    def __init__(self):
        super(model_ResNet_16, self).__init__()
        
        self.conv1 = conv_block(1, 16)
        self.conv2 = conv_block(16, 32, pool=True)
        self.res1 = conv_block(32, 32)
        
        self.conv3 = conv_block(32, 64, pool=True)
        self.res2 = conv_block(64, 64)

        self.conv4 = conv_block(64, 64, pool=True)
        self.res3 = conv_block(64, 64)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*2*2,3)#
            )
        self.ReLu = nn.ReLU()
        
    def forward(self, x):
      # In : (1,16,16)
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.ReLu(self.res1(out) + out)

      # In : (32,8,8)
        out = self.conv3(out)
        out = self.ReLu(self.res2(out) + out)

      # In : (64,4,4)
        out = self.conv4(out)
        out = self.ReLu(self.res3(out) + out)
        
        out = self.classifier(out)
        return out




### For input 32x32

In [ ]:
class model_CNN1_32(nn.Module):
    def __init__(self):
        super(model_CNN1_32, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 8, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(8),
          nn.ReLU(),
          
          # In : (8,16,16)
          nn.Conv2d(8, 16, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(),

          # In : (16,8,8)
          nn.Conv2d(16, 8,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(8),
          nn.ReLU(), 
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,8*4*4)
          nn.Linear(8*4*4, 3)
      )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN2_32(nn.Module):
    def __init__(self):
        super(model_CNN2_32, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 16, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          
          # In : (16,16,16)
          nn.Conv2d(16, 32, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),

          # In : (32,8,8)
          nn.Conv2d(32, 16,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(), 
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,16*4*4)
          nn.Linear(16*4*4, 128),
          nn.ReLU(),
          nn.Linear(128, 3)
          )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN3_32(nn.Module):
    def __init__(self):
        super(model_CNN3_32, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (8,30,30)
          nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (16,14,14)
          nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,12,12)
          nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2)
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,8*6*6)
          nn.Linear(8*5*5, 3)
          )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN4_32(nn.Module):
    def __init__(self):
        super(model_CNN4_32, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,30,30)
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (32,14,14)
          nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          # In : (32,12,12)
          nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2)
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,16*6*6)
          nn.Linear(16*5*5, 128),
          nn.ReLU(),
          nn.Linear(128, 3)
      )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN4_32_dropout(nn.Module):
    def __init__(self):
        super(model_CNN4_32_dropout, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,30,30)
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (32,14,14)
          nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          # In : (32,12,12)
          nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2)
          )
        self.classifier = nn.Sequential(
          # In : (batch_size,16*6*6)
          nn.Linear(16*5*5, 128),
          torch.nn.Dropout(0.5),
          nn.ReLU(),
          nn.Linear(128, 3)
      )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_VGGNet_32_dropout(nn.Module):
    def __init__(self):
        super(model_VGGNet_32_dropout, self).__init__()
        self.features = nn.Sequential(
            # In : (1,32,32)
            nn.Conv2d(1, 8, 3, padding=1),  # Conv1
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding=1),  # Conv2
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool1

             # In : (8,16,16)
            nn.Conv2d(8, 16, 3, padding=1),  # Conv3
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),  # Conv4
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,8,8)
            nn.Conv2d(16, 32, 3, padding=1),  # Conv5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # Conv6
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,4,4)
            nn.Conv2d(32, 64, 3, padding=1),  # Conv7
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool4

            # In : (64,2,2)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv9
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv10
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(2 * 2 * 64, 128),
            torch.nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return out
class model_VGGNet_32(nn.Module):
    def __init__(self):
        super(model_VGGNet_32, self).__init__()
        self.features = nn.Sequential(
            # In : (1,32,32)
            nn.Conv2d(1, 8, 3, padding=1),  # Conv1
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding=1),  # Conv2
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool1

             # In : (8,16,16)
            nn.Conv2d(8, 16, 3, padding=1),  # Conv3
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),  # Conv4
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,8,8)
            nn.Conv2d(16, 32, 3, padding=1),  # Conv5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # Conv6
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,4,4)
            nn.Conv2d(32, 64, 3, padding=1),  # Conv7
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool4

            # In : (64,2,2)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv9
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv10
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(2 * 2 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return out


class model_ResNet_32(nn.Module):
    def __init__(self):
        super(model_ResNet_32, self).__init__()
        
        self.conv1 = conv_block(1, 16)
        self.conv2 = conv_block(16, 32, pool=True)
        self.res1 = conv_block(32, 32)
        
        self.conv3 = conv_block(32, 64, pool=True)
        self.res2 = conv_block(64, 64)

        self.conv4 = conv_block(64, 64, pool=True)
        self.res3 = conv_block(64, 64)

        self.conv5 = conv_block(64, 64, pool=True)
        self.res4 = conv_block(64, 64)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*2*2,3)#
            )
        self.ReLu = nn.ReLU()
        
    def forward(self, x):
      # In : (1,32,32)
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.ReLu(self.res1(out) + out)


      # In : (32,16,16)
        out = self.conv3(out)
        out = self.ReLu(self.res2(out) + out)
 
      # In : (64,8,8)
        out = self.conv4(out)
        out = self.ReLu(self.res3(out) + out)

      # In : (64,4,4)
        out = self.conv5(out)
        out = self.ReLu(self.res4(out) + out)
        
        out = self.classifier(out)
        return out




### For input 32x32 & Voxel

In [ ]:
class model_CNN4_32_Voxel_dropout(nn.Module):
    def __init__(self):
        super(model_CNN4_32_Voxel_dropout, self).__init__()
    
        self.features_heatmap = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          # In : (16,30,30)
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (32,14,14)
          nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          # In : (32,12,12)
          nn.Conv2d(32, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2)
          )
        
        self.features_voxels = nn.Sequential(
          # In : (1,26,26,26)
          nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,24,24,24)
          nn.Conv3d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(32),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2),
          
          # In : (32,11,11,11)
          nn.Conv3d(32, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(16),
          nn.ReLU(),
          # In : (32,9,9,9)
          nn.Conv3d(16, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2)

          # Out : (8,3,3,3)
          )
        
        self.classifier = nn.Sequential(
          # In : (batch_size,16*6*6)
          nn.Linear(8*5*5 + 8*3*3*3, 128),
          torch.nn.Dropout(0.5),
          nn.ReLU(),
          nn.Linear(128, 1)
      )

    def forward(self, x_heatmap, x_voxel):
      heatmap_features = self.features_heatmap(x_heatmap)
      voxel_features = self.features_voxels(x_voxel)

      features = torch.cat((heatmap_features.view(x_heatmap.shape[0],-1),voxel_features.view(x_voxel.shape[0],-1)),dim=1)

      out = self.classifier(features)
      return(out)

class model_VGGNet_32_Voxel_dropout(nn.Module):
    def __init__(self):
        super(model_VGGNet_32_Voxel_dropout, self).__init__()
        self.features_heatmap = nn.Sequential(
            # In : (1,32,32)
            nn.Conv2d(1, 8, 3, padding=1),  # Conv1
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding=1),  # Conv2
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool1

             # In : (8,16,16)
            nn.Conv2d(8, 16, 3, padding=1),  # Conv3
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),  # Conv4
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,8,8)
            nn.Conv2d(16, 32, 3, padding=1),  # Conv5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # Conv6
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,4,4)
            nn.Conv2d(32, 64, 3, padding=1),  # Conv7
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool4

            # In : (64,2,2)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv9
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1),  # Conv10
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.features_voxels = nn.Sequential(
          # In : (1,26,26,26)
          nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,24,24,24)
          nn.Conv3d(16, 32, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(32),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2),
          
          # In : (32,11,11,11)
          nn.Conv3d(32, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(16),
          nn.ReLU(),
          # In : (32,9,9,9)
          nn.Conv3d(16, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2)

          # Out : (8,3,3,3)
          )
        

        self.classifier = nn.Sequential(
            nn.Linear(2*2*32 + 8*3*3*3, 128),
            torch.nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x_heatmap, x_voxel):
      heatmap_features = self.features_heatmap(x_heatmap)
      voxel_features = self.features_voxels(x_voxel)

      features = torch.cat((heatmap_features.view(x_heatmap.shape[0],-1),voxel_features.view(x_voxel.shape[0],-1)),dim=1)

      out = self.classifier(features)
      return out

### For input 64x64

In [ ]:
class model_CNN1_64(nn.Module):
    def __init__(self):
        super(model_CNN1_64, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,64,64)
          nn.Conv2d(1, 16, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(8),
          nn.ReLU(),  
          
          # In : (16,32,32)
          nn.Conv2d(16, 32, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          
          # In : (32,16,16)
          nn.Conv2d(32, 16, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU(),

          # In : (16,8,8)
          nn.Conv2d(16, 8,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(8),
          nn.ReLU()
          )
        
        self.classifier = nn.Sequential(
          # In : (batch_size,8*4*4)
          nn.Linear(8*4*4, 3)
      )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN2_64(nn.Module):
    def __init__(self):
        super(model_CNN2_64, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,64,64)
          nn.Conv2d(1, 32, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          
          # In : (16,32,32)
          nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(64),
          nn.ReLU(),

          # In : (64,16,16)
          nn.Conv2d(64, 32,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(32),
          nn.ReLU(), 

          # In : (32,8,8)
          nn.Conv2d(32, 16,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(16),
          nn.ReLU()         
          )
        
        self.classifier = nn.Sequential(
          # In : (batch_size,16*4*4)
          nn.Linear(16*4*4, 128),
          nn.ReLU(),
          nn.Linear(128, 3)
      )
        
    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)

class model_CNN3_64(nn.Module):
    def __init__(self):
        super(model_CNN3_64, self).__init__()
    
        self.features = nn.Sequential(
          # In : (1,64,64)
          nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (8,62,62)
          nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (16,30,30)
          nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,28,28)
          nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),

          # In : (16,13,13)
          nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          # In : (16,11,11)
          nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2)
          )
        
        self.classifier = nn.Sequential(
          # In : (batch_size,8*6*6)
          nn.Linear(8*6*6, 3)
      )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return(out)


class model_VGGNet_64(nn.Module):
    def __init__(self):
        super(model_VGGNet_64, self).__init__()
        self.features = nn.Sequential(
            # In : (1,64,64)
            nn.Conv2d(1, 8, 3, padding=1),  # Conv1
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding=1),  # Conv2
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool1
            
            # In : (8,32,32)
            nn.Conv2d(8, 16, 3, padding=1),  # Conv3
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),  # Conv4
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,16,16)
            nn.Conv2d(16, 32, 3, padding=1),  # Conv5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # Conv6
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,8,8)
            nn.Conv2d(32, 64, 3, padding=1),  # Conv7
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool4

            # In : (64,4,4)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv9
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv10
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool5

            # In : (64,2,2)
            nn.Conv2d(64, 64, 3, padding=1),  # Conv11
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # Conv12
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(2 * 2 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
      inter = self.features(x)
      out = self.classifier(inter.view(x.shape[0],-1))
      return out

class model_ResNet_64(nn.Module):
    def __init__(self):
        super(model_ResNet_64, self).__init__()
        
        self.conv1 = conv_block(1, 16)
        self.conv2 = conv_block(16, 32, pool=True)
        self.res1 = conv_block(32, 32)
        
        self.conv3 = conv_block(32, 64, pool=True)
        self.res2 = conv_block(64, 64)

        self.conv4 = conv_block(64, 64, pool=True)
        self.res3 = conv_block(64, 64)

        self.conv5 = conv_block(64, 64, pool=True)
        self.res4 = conv_block(64, 64)

        self.conv6 = conv_block(64, 64, pool=True)
        self.res5 = conv_block(64, 64)        
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*2*2, 3)#
            )
        self.ReLu = nn.ReLU()
        
    def forward(self, x):
      # In : (1,64,64)
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.ReLu(self.res1(out) + out)

      # In : (32,32,32)
        out = self.conv3(out)
        out = self.ReLu(self.res2(out) + out)

      # In : (64,16,16)
        out = self.conv4(out)
        out = self.ReLu(self.res3(out) + out)

      # In : (64,8,8)
        out = self.conv5(out)
        out = self.ReLu(self.res4(out) + out)

      # In : (64,4,4)
        out = self.conv6(out)
        out = self.ReLu(self.res5(out) + out)
        
        out = self.classifier(out)
        return out




### Test model type


In [ ]:
########### Heatmap Networks
class model_heatmap1(nn.Module):
    def __init__(self, channels = 8):
        super(model_heatmap1, self).__init__()

        self.c = channels
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, self.c, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(self.c),
          nn.ReLU(),
          
          # In : (c,16,16)
          nn.Conv2d(self.c, 2*self.c, kernel_size=2, stride=2, padding=0, bias=True),
          nn.BatchNorm2d(2*self.c),
          nn.ReLU(),

          # In : (2c,8,8)
          nn.Conv2d(2*self.c, self.c,  kernel_size=2, stride=2, padding=0, bias=True),
          nn.ReLU(), 
          )
          # Out : (batch_size,c,4,4)
    def forward(self, x):
      heat_features = self.features(x)
      return heat_features


class model_heatmap2(nn.Module):
    def __init__(self, channels = 8):
        super(model_heatmap2, self).__init__()

        self.c = channels
    
        self.features = nn.Sequential(
          # In : (1,32,32)
          nn.Conv2d(1, self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(self.c),
          nn.ReLU(),
          # In : (c,30,30)
          nn.Conv2d(self.c, 2*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(2*self.c),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
          
          # In : (2c,14,14)
          nn.Conv2d(2*self.c, 2*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(2*self.c),
          nn.ReLU(),
          # In : (2c,12,12)
          nn.Conv2d(2*self.c, 2*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm2d(2*self.c),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),

          # In : (c,6,6)
          nn.Conv2d(2*self.c, self.c, kernel_size=2, stride=1, padding=0, bias=True),
          nn.ReLU(),

          # Out : (batch_size,c,4,4)
          )
        
    def forward(self, x):
      heat_features = self.features(x)
      return heat_features


class model_heatmapVGG(nn.Module):
    def __init__(self, channels=8):
        super(model_heatmapVGG, self).__init__()
        
        self.c = channels

        self.features = nn.Sequential(
            # In : (1,32,32)
            nn.Conv2d(1, self.c, 3, padding=1),  # Conv1
            nn.BatchNorm2d(self.c),
            nn.ReLU(),
            nn.Conv2d(self.c, self.c, 3, padding=1),  # Conv2
            nn.BatchNorm2d(self.c),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool1

             # In : (8,16,16)
            nn.Conv2d(self.c, 2*self.c, 3, padding=1),  # Conv3
            nn.BatchNorm2d(2*self.c),
            nn.ReLU(),
            nn.Conv2d(2*self.c, 2*self.c, 3, padding=1),  # Conv4
            nn.BatchNorm2d(2*self.c),
            nn.ReLU(),

            # In : (8,16,16)
            nn.Conv2d(2*self.c, 2*self.c, 3, padding=1),  # Conv5
            nn.BatchNorm2d(2*self.c),
            nn.ReLU(),
            nn.Conv2d(2*self.c, 4*self.c, 3, padding=1),  # Conv6
            nn.BatchNorm2d(4*self.c),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool2

            # In : (16,8,8)
            nn.Conv2d(4*self.c, 4*self.c, 3, padding=1),  # Conv7
            nn.BatchNorm2d(4*self.c),
            nn.ReLU(),
            nn.Conv2d(4*self.c, 2*self.c, 3, padding=1),  # Conv8
            nn.BatchNorm2d(2*self.c),
            nn.ReLU(),

            # In : (16,8,8)
            nn.Conv2d(2*self.c, 2*self.c, 3, padding=1),  # Conv9
            nn.BatchNorm2d(2*self.c),
            nn.ReLU(),
            nn.Conv2d(2*self.c, self.c, 3, padding=1),  # Conv10
            nn.BatchNorm2d(self.c),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Pool3

            # In : (32,4,4)
            nn.Conv2d(self.c, self.c, 3, padding=1),  # Conv11
            nn.BatchNorm2d(self.c),
            nn.ReLU(),
            nn.Conv2d(self.c, self.c, 3, padding=1),  # Conv12
            nn.ReLU(),

            # Out : (c,4,4)
        )


    def forward(self, x):
      heat_features = self.features(x)
      return heat_features

# either = in -1
# or = (in -2)//2 +1

class model_heatmapResNet(nn.Module):
    def __init__(self, channels=8):
        super(model_heatmapResNet, self).__init__()

        self.c = channels

        # In : (1,32,32)
        self.conv1 = conv_block(1, 2*self.c)
        self.bn1 = nn.BatchNorm2d(2*self.c)
        self.conv2 = conv_block(2*self.c, 4*self.c, pool=True)
        self.bn2 = nn.BatchNorm2d(4*self.c)
        self.res1 = conv_block(4*self.c, 4*self.c)
        
        self.conv3 = conv_block(4*self.c, 4*self.c, pool=True)
        self.bn3 = nn.BatchNorm2d(4*self.c)
        self.res2 = conv_block(4*self.c, 4*self.c)

        self.conv4 = conv_block(4*self.c, self.c, pool=True)
        self.bn4 = nn.BatchNorm2d(self.c)
        self.res3 = conv_block(self.c, self.c)

        self.ReLu = nn.ReLU()
        
    def forward(self, x):
      # In : (1,32,32)
        out = self.bn1(self.conv1(x))
        out = self.bn2(self.conv2(out))
        out = self.ReLu(self.res1(out) + out)

      # In : (4c,16,16)
        out = self.bn3(self.conv3(out))
        out = self.ReLu(self.res2(out) + out)
 
      # In : (4c,8,8)
        out = self.bn4(self.conv4(out))
        out = self.ReLu(self.res3(out) + out)

        # Out : (c,4,4)
        return out



########### Voxel Networks
class model_Voxel1(nn.Module):
    def __init__(self, channels = 8):
        super(model_Voxel1, self).__init__()

        self.c = channels

        self.features_voxels = nn.Sequential(
          # In : (1,26,26,26)
          nn.Conv3d(1, 2*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(2*self.c),
          nn.ReLU(),
          # In : (16,24,24,24)
          nn.Conv3d(2*self.c, 4*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(4*self.c),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2),
          
          # In : (32,11,11,11)
          nn.Conv3d(4*self.c, 2*self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.BatchNorm3d(2*self.c),
          nn.ReLU(),
          # In : (32,9,9,9)
          nn.Conv3d(2*self.c, self.c, kernel_size=3, stride=1, padding=0, bias=True),
          nn.ReLU(),
          nn.MaxPool3d(kernel_size=2,stride=2)

          # Out : (8,3,3,3)
          )
        

    def forward(self, x_voxel):
      voxel_features = self.features_voxels(x_voxel)

      return voxel_features



class model_VoxelVGG(nn.Module):
    def __init__(self, channels=8):
        super(model_VoxelVGG, self).__init__()
        self.c = channels
        self.features_voxels = nn.Sequential(
            # In : (1,26,26,26)
            nn.Conv3d(1, self.c, 3, padding=1),  # Conv1
            nn.BatchNorm3d(self.c),
            nn.ReLU(),
            nn.Conv3d(self.c, self.c, 3, padding=1),  # Conv2
            nn.BatchNorm3d(self.c),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2,stride=2),  # Pool1

            # In : (8,13,13,13)
            nn.Conv3d(self.c, 2*self.c, 3, padding=1),  # Conv3
            nn.BatchNorm3d(2*self.c),
            nn.ReLU(),
            nn.Conv3d(2*self.c, 2*self.c, 3, padding=1),  # Conv4
            nn.BatchNorm3d(2*self.c),
            nn.ReLU(),

            # In : (8,13,13,13)
            nn.Conv3d(2*self.c, 2*self.c, 3, padding=1),  # Conv5
            nn.BatchNorm3d(2*self.c),
            nn.ReLU(),
            nn.Conv3d(2*self.c, 4*self.c, 3, padding=1),  # Conv6
            nn.BatchNorm3d(4*self.c),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2,stride=2),  # Pool2
            
            # In : (8,6,6,6)
            nn.Conv3d(4*self.c, 4*self.c, 3, padding=1),  # Conv7
            nn.BatchNorm3d(4*self.c),
            nn.ReLU(),
            nn.Conv3d(4*self.c, 2*self.c, 3, padding=1),  # Conv8
            nn.BatchNorm3d(2*self.c),
            nn.ReLU(),
            
            # In : (8,6,6,6)
            nn.Conv3d(2*self.c, 2*self.c, 3, padding=1),  # Conv9
            nn.BatchNorm3d(2*self.c),
            nn.ReLU(),
            nn.Conv3d(2*self.c, self.c, 3, padding=1),  # Conv10
            nn.BatchNorm3d(self.c),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2,stride=2),  # Pool3

            # In : (8,3,3,3)
            nn.Conv3d(self.c, self.c, 3, padding=1),  # Conv11
            nn.BatchNorm3d(self.c),
            nn.ReLU(),
            nn.Conv3d(self.c, self.c, 3, padding=1),  # Conv12
            nn.ReLU(),

            # Out : : (8,3,3,3)
        )

    def forward(self, x_voxel):
      voxel_features = self.features_voxels(x_voxel)
      return voxel_features





#### Combining :
class model_heat_vox(nn.Module):
    def __init__(self, heat_model=1, vox_model=1, batchNorm=True, dropout=0.5, channels=8):
        super(model_heat_vox, self).__init__()

        self.heat_model=heat_model # 4 poss
        self.vox_model=vox_model # 2 poss
        self.dropout=dropout # 3 poss
        self.c = channels


        if heat_model == "1":
          self.features_heatmap = model_heatmap1(channels=self.c)
        elif heat_model == "2":
          self.features_heatmap = model_heatmap2(channels=self.c)
        elif heat_model == "VGG":
          self.features_heatmap = model_heatmapVGG(channels=self.c)
        elif heat_model == "ResNet":
          self.features_heatmap = model_heatmapResNet(channels=self.c)
        
        if vox_model == "1":
          self.features_voxels = model_Voxel1(channels=self.c)
        elif vox_model == "VGG":
          self.features_voxels = model_VoxelVGG(channels=self.c)
        
        # Classfier : (not defined in sequential to make sure dropout is off when model.eval()
        self.drop_layer = nn.Dropout(p=dropout)

        self.fc1 = nn.Linear(channels*(4*4 + 3*3*3), 128)
        self.fc2 = nn.Linear(128, 1)


        # self.classifier = nn.Sequential(
        #     nn.Linear(2*2*32 + 8*3*3*3, 128), # 344
        #     torch.nn.Dropout(0.5),
        #     nn.ReLU(),
        #     nn.Linear(128, 1)
        # )

    def forward(self, x_heatmap, x_voxel):
      heatmap_features = self.features_heatmap(x_heatmap)
      voxel_features = self.features_voxels(x_voxel)

      features = torch.cat((heatmap_features.view(x_heatmap.shape[0],-1),voxel_features.view(x_voxel.shape[0],-1)),dim=1)

      out = self.fc1(features)
      out = self.drop_layer(out)
      out = nn.ReLU()(out)
      out = self.fc2(out)
      # out = self.classifier(features)
      return out

## Iterables

### Heatmap only

In [ ]:
# "Easy" (typical) hyper params:
learning_rates = [0.0001,0.01]#,0.001,0.0001]
batch_sizes=[128,64]#,128]#,128]
num_epochs_list=[30]#20,30

# Less intuitve params : e.g network type, data type and should be coordinated 
# (+ maybe: resizing image vs using appropriate network)
dataset = ["32_8","32_16","32_32","64_8","64_16","16_16","full"]

models_16 = ["model_CNN1_16","model_CNN2_16","model_CNN3_16","model_VGGNet_16","model_ResNet_16"]
models_32 = ["model_CNN4_32_dropout"]#["model_CNN4_32_dropout","model_CNN2_32","model_CNN4_32","model_VGGNet_32_dropout"]#"model_ResNet_32"#["model_CNN1_32","model_CNN3_32","model_CNN4_32","model_VGGNet_32","model_ResNet_32"]#"model_CNN2_32"
models_64 = ["model_CNN1_64","model_CNN2_64","model_CNN3_64","model_VGGNet_64","model_ResNet_64"]

transf_list =[transform2_heatmap,None]#[transform1,transform2,None]
    # transforms.Pad(4),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomCrop(32),
    # transforms.ToTensor()])

### Heatmap + voxels

In [ ]:
# "Easy" (typical) hyper params:
learning_rates = [0.0001]#[0.01,0.001,0.0001]
batch_sizes=[64]#[64,128]#,128]
num_epochs_list=[30]#20,30

# Less intuitve params : e.g network type, data type and should be coordinated 
# (+ maybe: resizing image vs using appropriate network)


models_heatmap_voxels = ["model_VGGNet_32_Voxel_dropout","model_CNN4_32_Voxel_dropout"]

models_heat = ["ResNet","VGG"]#["2","VGG"]#,"ResNet"] # To run : 1,2
models_voxels = ["VGG"] # 1
dropouts =[0,0.1]#0.3
channels = [8] # 4

# knife_spoon_living
# tensor(819.3264, dtype=torch.float64)
# tensor(2452.4735, dtype=torch.float64)
# tensor(4.0342, dtype=torch.float64)
# tensor(159.1961, dtype=torch.float64)

#knife_spoon)living_hull
# tensor(826.7405, dtype=torch.float64)
# tensor(2492.3940, dtype=torch.float64)
# tensor(12.3799, dtype=torch.float64)
# tensor(260.7471, dtype=torch.float64)

t_heat = transforms.Compose([transforms.RandomApply(transforms=[
                         transforms.RandomVerticalFlip(p=0.2),
                         transforms.RandomHorizontalFlip(p=0.9),
                        #  AddGaussianNoise(0, 50),
                        #  transforms.Compose([transforms.RandomCrop(size=(24, 24)), transforms.Pad(4)]),
                         transforms.RandomAffine(degrees=(-1, 1), translate=(0.1, 0.1), scale=(0.95, 0.95)),
                         transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 3))
                         ], p=0.6),
                        transforms.Normalize(mean=826.7405,std=2492.3940)])
t_vox = transforms.Compose([transforms.RandomApply(transforms=[
                         RandomFlip_Voxel(p=0.8, dim=3),
                         RandomFlip_Voxel(p=0.8, dim=1),
                         RandomFlip_Voxel(p=0.2, dim=2)], p=0.6),
                        #  AddGaussianNoise(0, 50)], p=0.6),
                         Normalize_Voxel(mean=12.3799,std=260.7471)])



#t8
transf_list_heatmap_voxels =[[t_heat,t_vox]]#[[t1,t2] for t1 in transf_list for t2 in transf_list_voxels]
transf = [t_heat,t_vox]
# transf_list_voxels = [transform1_voxel_1,transform1_voxel_2,None,transform1_voxel_3]
# transf_list_heatmap_voxels =[[t1,t2] for t1 in transf_list for t2 in transf_list_voxels]

# Training Part - CNN based Classification

## Heatmap only

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
path_train,path_val = '/content/drive/MyDrive/mmWaveData/Dataset_shovel_rake/mmwaveRA.csv','/content/drive/MyDrive/mmWaveData/Dataset_shovel_rake_val/mmwaveRA.csv'
num_epochs = 30
loss_function =nn.BCEWithLogitsLoss()# nn.CrossEntropyLoss()

def accu(res_batch,labels):
  res = torch.argmax(res_batch, dim=1)
  eq = torch.eq(res,labels)
  true = torch.count_nonzero(eq)
  accu = true/len(res) 
  return accu.item()


for model_type in models_32:
  for lr in learning_rates:
    for batch_size in batch_sizes:
      for transf in transf_list:
        train_loader, validation_loader = get_data_loaders_from(path_train,path_val,batch_size=32,transforms_heatmap=transf)

        ### Create Model
        print(model_type)
        if model_type=="model_CNN1_32":
          model = model_CNN1_32()
        elif model_type=="model_CNN2_32":
          model = model_CNN2_32()
        elif model_type=="model_CNN3_32":
          model = model_CNN3_32()
        elif model_type=="model_CNN4_32":
          model = model_CNN4_32()
        elif model_type=="model_CNN4_32_dropout":
          model = model_CNN4_32_dropout()
        elif model_type=="model_VGGNet_32":
          model = model_VGGNet_32()
        elif model_type=="model_ResNet_32":
          model = model_ResNet_32()
        elif model_type=="model_VGGNet_32_dropout":
          model = model_VGGNet_32_dropout()
        # model = model_knife_spoon_ROI2()
        params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        ### Define Optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        ### Log run config to WandB
        config = {"num_epochs": num_epochs,
                  "lr": lr,
                  "batch_size": batch_size,
                  "transforms": transf,#data_type,
                  "model_type": model_type,
                  "num_params": params,
                  "datatype": "heatmap"}

        wandb.init(project="mmmwave_Shovel_Rake", config=config)
        wandb.watch(model)
        print("Total number of parameters is: {}".format(params))

        ### Load Data:
        train_loader, validation_loader = get_data_loaders_from(path_train,path_val,batch_size=batch_size,transforms=transf)
      
        ### Train
        # send model to device
        model = model.to(device)

        # to keep track of losses
        loss_epoch_train = []
        loss_epoch_val = []
        loss_epoch_test = []
        train_step = 0
        val_step = 0

        print(config)
        for epoch in range(num_epochs): 
                correct = 0.
                total = 0.
                ############## Train ##############
                model.train()
                accs_train = []    
                loss_train = 0 # KL regularizer loss during this training epoch
                for b_idx, (data, labels) in enumerate(train_loader):
                  print(b_idx)
                  data = data.to(device, dtype=torch.float)
                  labels = labels.to(device,dtype=torch.float)
                  # reset gradient
                  optimizer.zero_grad()
                  # get prediction 
                  res_batch = model(data)#.float())
                  # get loss
                  loss = loss_function(res_batch,labels)#res_batch, labels)
                  # Only use the "total" loss for optimizing...
                  loss.backward()
                  optimizer.step()
                  # accs_train.append(accu(res_batch,labels))

                  # Keep loss for plotting eventually
                  loss_train += loss

                  # Log to WandB
                  # wandb.log({"train_loss": loss.item(), "train_step": train_step})
                  train_step+=1


                  #### NEW ####

                  outputs = torch.sigmoid(res_batch).cpu()        #<--- since you use BCEWithLogitsLoss
                  #round up and down to either 1 or 0
                  predicted = np.round(np.asarray(outputs.detach())) #keep in mind that np.round() is a round to even function
                  total += labels.size(0)
                  #calculate how many images were correctly classified
                  correct += (predicted == np.asarray(labels.cpu())).sum().item()
                accuracy_train = 100 * correct / total

                ## ______###
          
                loss_train /= b_idx

                ############## Eval ##############
                model.eval()
                accs_val = []    
                loss_val = 0 # KL regularizer loss during this training epoch
                #loss_list.append(loss)
                
                labs = []
                preds = []
                correct = 0.
                total = 0.
                with torch.no_grad():
                  for b_idx, (data, labels) in enumerate(validation_loader):
                    print(b_idx)
                    data = data.to(device, dtype=torch.float)
                    labels = labels.to(device,dtype=torch.float)

                    # get prediction 
                    res_batch = model(data)
                  # Compute loss and accuracy
                    loss = loss_function(res_batch,labels)

                    # accs_val.append(accu(res_batch,labels))

                # Keep loss for plotting eventually?
                    loss_val += loss


                    # preds.extend(torch.argmax(res_batch, dim=1).cpu().detach().numpy())
                    # labs.extend(labels.cpu().detach().numpy())
                  # acc = [0,0,0]
                  # preds = torch.argmax(res_batch, dim=1)
                  # for c in range(len(acc)):
                  #     acc[c] = (((preds == labels) * (labels == c)).float().sum()) / (max((labels == c).sum(), 1))
                  # # Log to WandB
                  # wandb.log({"val_loss": loss.item(), "val_step": val_step,"Knife_acc_val": acc[0].item(),"Spoon_acc_val": acc[1].item(),"Nothing_acc_val": acc[2].item()})
                  val_step += 1

                                    ### NEW ####

                  outputs = torch.sigmoid(res_batch).cpu()        #<--- since you use BCEWithLogitsLoss
                  #round up and down to either 1 or 0
                  predicted = np.round(outputs.detach().numpy()) #keep in mind that np.round() is a round to even function
                  total += labels.size(0)
                  #calculate how many images were correctly classified

                  correct += (predicted == np.asarray(labels.cpu())).sum().item()
                accuracy_val = 100 * correct / total
                
                ### ______###
                loss_val /= b_idx

                # labs = np.array(labs)
                # preds = np.array(preds)
                # acc = [0,0,0]
                # for c in range(len(acc)):
                #     acc[c] = (np.sum(((preds == labs) * (labs == c)))) / (max(np.sum((labs == c)), 1))


                loss_epoch_train.append(loss_train)
                loss_epoch_val.append(loss_val)
                # loss_epoch_test.append(loss_test)
                # wandb.log({"train_loss": loss_train,"val_loss": loss_val,"acc_train":accuracy_train,"acc_val":accuracy_val})#""Knife_acc_val": acc[0].item(),"Spoon_acc_val": acc[1].item(),"Nothing_acc_val": acc[2].item()})
                
                print('Epoch {0} :::::: Train Loss: {1}'.format(epoch, loss_train))#len(train_loader.dataset)))
                print('              :: Train Accuracy: {0}'.format(np.mean(accs_train)))#len(train_loader.dataset)))
                print('              ____________ ')
                print('              :: Val Loss: {1}'.format(epoch, loss_val))#len(train_loader.dataset)))
                print('              :: Val Accuracy: {0}'.format(np.mean(accs_val)))#len(train_loader.dataset)))



    # # plotting the train loss 
    # plt.plot(loss_epoch_train , label = "Train")
    # # plotting the Val loss 
    # plt.plot(loss_epoch_val , label = "Val")

    # plt.plot(loss_epoch_test , label = "test")


    # # # plotting the test loss 
    # # plt.plot(loss_epoch_test , label = "test")


    # plt.xlabel('epoch')
    # # Set the y axis label of the current axis.
    # plt.ylabel('loss')
    # # Set a title of the current axes.
    # plt.title('Two or more lines on same plot with suitable legends ')
    # # show a legend on the plot
    # plt.legend()
    # # Display a figure.
    # plt.show()

torch.save(model, "/content/drive/MyDrive/mmWaveData/model_rake_shovel.pt")

model_CNN4_32_dropout


train_loss,0.37469
val_loss,0.61766
acc_train,248.83501
acc_val,217.10526
_runtime,1023
_timestamp,1627901888
_step,99


train_loss,█▇▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_loss,█▇▄▃▃▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▂▃▃▃▃
acc_train,▁▂▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇██▇██████
acc_val,▁▄▅▄▅▆▅█▇▆▇▅▇▅▇█▄▆▇▇▇▇▅▃▅▄▄▆▇▅▄▅▅▅▆▅▅▅▅▅
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Total number of parameters is: 70515
{'num_epochs': 30, 'lr': 0.0001, 'batch_size': 128, 'transforms': Compose(
    RandomVerticalFlip(p=0.5)
    RandomHorizontalFlip(p=0.5)
), 'model_type': 'model_CNN4_32_dropout', 'num_params': 70515}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
Epoch 0 :::::: Train Loss: 0.7144678831100464
              :: Train Accuracy: nan
              ____________ 
              :: Val Loss: 0.8190261721611023
              :: Val Accuracy: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
Epoch 1 :::::: Train Loss: 0.7029471397399902
              :: Train Accuracy: nan
              ____________ 
              :: Val Loss: 0.8028499484062195
              :: Val Accuracy: nan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
Epoch 2 :::::: Train Loss: 0.6887046694755554
              :: Train Accuracy: nan
              ____________ 
              :: Val Loss: 0.7809669971466064
              :: Val Accuracy: nan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
Epoch 3 :::::: Train Loss: 0.6687943339347839
              :: Train Accuracy: nan
              ____________ 
              :: Val Loss: 0.746647298336029
              :: Val Accuracy: nan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
Epoch 4 :::::: Train Loss: 0.6472971439361572
              :: Train Accuracy: nan
              ____________ 
      

## Heatmap + voxel

In [ ]:
path_train,path_val = '/content/drive/MyDrive/mmWaveData/knife_spoon_living_hull/mmwaveRA.csv','/content/drive/MyDrive/mmWaveData/knife_spoon_living_hull_test/mmwaveRA.csv'
num_epochs = 30
loss_function = nn.BCEWithLogitsLoss()#nn.CrossEntropyLoss()


lr= 0.0001

def metrics(res_batch,labels):
  pred = np.array(res_batch)
  predicted = (pred>0.5)
  # pred_false = (pred<0.5)
  TP = (predicted == labels * pred == 0).sum()
  TN = (predicted == labels * pred == 1).sum()
  FP = (predicted != labels * pred == 0).sum()
  FN = (predicted != labels * pred == 1).sum()
  # TP_func = (pred_true == labels).sum()
  # TN_func = (pred_false == labels).sum()
  # FP_func = (pred_true != labels).sum()
  # FN_func = (pred_false != labels).sum()

  # precision = TP/(TP + FP)
  # recall = TP/(TP + FN)
  # F1 = 2*precision*recall/(precision+recall)
  # acc = (TP + TN) /(TP + TN + FN + FP)
  return TP,TN,FP,FN#precision,recall,F1, acc


for heat_type in models_heat:
  for vox_type in models_voxels:
    for batch_size in batch_sizes:
      for dropout in dropouts:
        for c in channels :
          torch.manual_seed(0)
          np.random.seed(0)
          random.seed(0)

          train_loader, validation_loader = get_data_loaders_from(path_train,path_val,batch_size=batch_size,transforms_heatmap=transf[0],transforms_voxel=transf[1],datatype="heatmap_voxel")



          ### Create Model
          # print(model_type)
          # if model_type=="model_CNN4_32_Voxel_dropout":
          #   model = model_CNN4_32_Voxel_dropout()
          # elif model_type=="model_VGGNet_32_Voxel_dropout":
          #   model = model_VGGNet_32_Voxel_dropout()
          print("Heat type: " + heat_type +" ____ Vox type: " + vox_type)
          model = model_heat_vox(heat_model=heat_type, vox_model=vox_type, batchNorm=True, dropout=dropout, channels= c)
          params = sum(p.numel() for p in model.parameters() if p.requires_grad)

          ### Define Optimizer
          optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0.0001)

          ### Log run config to WandB
          config = {"num_epochs": num_epochs,
                    "lr": lr,
                    "batch_size": batch_size,
                    "transforms": transf,
                    "dropout":dropout,
                    "channels":c,
                    "heat_type": heat_type,
                    "vox_type": vox_type,
                    "num_params": params,
                    "batchnorm":True,
                    "datatype": "heatmap_voxel_hull"}
# Uncom
          wandb.init(project="mmmwaveKnife_Spoon_HeatVox", config=config)
          wandb.watch(model)
          print("Total number of parameters is: {}".format(params))
        
          ### Train
          # send model to device
          model = model.to(device, dtype=torch.float)

          # to keep track of losses
          loss_epoch_train = []
          loss_epoch_val = []
          loss_epoch_test = []
          train_step = 0
          val_step = 0
          print(config)
          for epoch in range(num_epochs): 
                  correct = 0.
                  total = 0.

                  ############## Train ##############
                  model.train()
                  accs_train = []    
                  loss_train = 0 
                  for b_idx, (data_heatmap,data_voxel, labels) in enumerate(train_loader):
                    print(b_idx)
                    data_heatmap = data_heatmap.to(device, dtype=torch.float)
                    data_voxel = data_voxel.to(device, dtype=torch.float)
                    labels = labels.to(device,dtype=torch.float)
                    # reset gradient
                    optimizer.zero_grad()
                    # get prediction 
                    res_batch = model(data_heatmap,data_voxel)
                    # get loss
                    loss = loss_function(res_batch.squeeze(1),labels)
                    # Use the "total" loss for optimizing...
                    loss.backward()
                    optimizer.step()
                    # accs_train.append(accu(res_batch,labels))

                    # Keep loss for plotting eventually
                    loss_train += loss


                    train_step+=1
                  loss_train /= b_idx






                  ############## Eval ##############
                  model.eval()
                  accs_val = []    
                  loss_val = 0 # KL regularizer loss during this training epoch
                  #loss_list.append(loss)
                  
                  labels_all = []
                  preds_all = []
                  correct = 0.
                  total = 0.
                  TP,TN,FP,FN= 0,0,0,0
                  with torch.no_grad():
                    for b_idx, (data_heatmap,data_voxel, labels) in enumerate(validation_loader):
                      data_heatmap = data_heatmap.to(device, dtype=torch.float)
                      data_voxel = data_voxel.to(device, dtype=torch.float)
                      labels = labels.to(device,dtype=torch.float)

                      # get prediction 
                      res_batch = model(data_heatmap, data_voxel)
                      # Compute loss and accuracy
  
                      loss = loss_function(res_batch.squeeze(1),labels)

                      # accs_val.append(accu(res_batch,labels))

                  # Keep loss for plotting eventually?
                      loss_val += loss

                      # preds_all.extend((res_batch.cpu().detach().numpy())
                      pred = (nn.Sigmoid()(res_batch.squeeze(1))).cpu().detach().numpy()
                      pred= np.array(pred)
                      predicted = pred >0.5
                      labels=labels.cpu().detach().numpy()

                      
                      TP += ((predicted == labels) * (predicted == 1)).sum()
                      TN += ((predicted == labels) * (predicted == 0)).sum()
                      FP += ((predicted != labels) * (predicted == 1)).sum()
                      FN += ((predicted != labels) * (predicted == 0)).sum()

                    # acc = [0,0,0]
                    # preds = torch.argmax(res_batch, dim=1)
                    # for c in range(len(acc)):
                    #     acc[c] = (((preds == labels) * (labels == c)).float().sum()) / (max((labels == c).sum(), 1))
                    # # Log to WandB
                    # wandb.log({"val_loss": loss.item(), "val_step": val_step,"Knife_acc_val": acc[0].item(),"Spoon_acc_val": acc[1].item(),"Nothing_acc_val": acc[2].item()})
                    val_step += 1

                  loss_val /= b_idx

         
                  precision = TP/(TP + FP)
                  recall = TP/(TP + FN)
                  F1 = 2*precision*recall/(precision+recall)
                  acc = (TP + TN) /(TP + TN + FN + FP)

                  # precision, recall, F1, acc =  metrics(preds_all,labels_all)

                  loss_epoch_train.append(loss_train)
                  loss_epoch_val.append(loss_val)
                  # loss_epoch_test.append(loss_test)
# Uncom
                  wandb.log({"train_loss": loss_train,"val_loss": loss_val,"Precision": precision,"recall": recall,"F1":F1, "Accuracy":acc})#,"Nothing_acc_val": acc[2].item()})
                  
                  print('Epoch {0} :::::: Train Loss: {1}'.format(epoch, loss_train))#len(train_loader.dataset)))
                  print('              ____________ ')
                  print('              :: Val Loss: {1}'.format(epoch, loss_val))#len(train_loader.dataset)))
                  print('              :: Val Accuracy: {0}'.format(acc))#len(train_loader.dataset)))
                  if (acc >0.73) or (F1 > 0.75):
                    torch.save(model, "/content/drive/MyDrive/mmWaveData/models/"+str(wandb.run.id)+"_"+str(epoch)+".pt")

Heat type: ResNet ____ Vox type: VGG


train_loss,0.15823
val_loss,0.78012
Precision,0.84762
recall,0.59333
F1,0.69804
Accuracy,0.71587
_runtime,850
_timestamp,1629317875
_step,29


train_loss,██▇▇▆▆▅▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_loss,▆▅▄▄▃▃▂▂▃▂▁▅▂▃▁▆▇▄▅▃█▄▇▄▇▆▃▃▇▄
Precision,█▄▁▁▁▁▂▅▆▅▅█▄▆▅▃▃▃▇▆▇▂▅▃▇▆▄▆▇▅
recall,▁▄▆▆█▆▇▆▅▆▆▃▇▅▆▇▇█▅▆▄▇▅▇▅▅▇▆▅▆
F1,▁▅▇▇█▇▇▇▆▇▇▅█▇▇▇██▆▇▆▇▇█▆▇██▆▇
Accuracy,▁▄▄▅▆▅▆▇▆▇▇▄█▆▇▆▇█▆▇▆▆▆▇▆▆▇█▆▇
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


Total number of parameters is: 169105
{'num_epochs': 30, 'lr': 0.0001, 'batch_size': 64, 'transforms': [Compose(
    RandomApply(
    p=0.6
    RandomVerticalFlip(p=0.2)
    RandomHorizontalFlip(p=0.9)
    RandomAffine(degrees=[-1.0, 1.0], translate=(0.1, 0.1), scale=(0.95, 0.95))
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 3))
)
    Normalize(mean=826.7405, std=2492.394)
), Compose(
    RandomApply(
    p=0.6
    RandomFlip_Voxel(p=0.8,dim=3)
    RandomFlip_Voxel(p=0.8,dim=1)
    RandomFlip_Voxel(p=0.2,dim=2)
)
    Normalize_Voxel(mean=12.3799,std=260.7471)
)], 'dropout': 0, 'channels': 8, 'heat_type': 'ResNet', 'vox_type': 'VGG', 'num_params': 169105, 'batchnorm': True, 'datatype': 'heatmap_voxel_hull'}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 0 :::::: Train Loss: 0.6698306798934937
              ____________ 
              :: Val Loss: 0.8428863883018494
              :: Val Accuracy: 0.5682656826568265
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 1 :::::: T

train_loss,0.12839
val_loss,0.80204
Precision,0.79856
recall,0.74
F1,0.76817
Accuracy,0.75277
_runtime,809
_timestamp,1629320141
_step,29


...

train_loss,█▇▇▆▅▅▅▄▄▄▃▃▃▃▂▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁
val_loss,▄▃▂▂▃▁▃▂▃▁▂▂▁▂▃▃▂▅▂▁▂▁█▂▄▄▃▄▄▄
Precision,▃▂▁▁▆▃▅▄▅▃▃▅▆▄▇▅█▇▆▇▄▅▆▆▆▆▄▆▆▄
recall,▁▂▆▆▂▇▃▄▂▆▆▆▅▇▄▇▆▃▇▆█▇▅▇▆▅▇▆▆█
F1,▁▂▆▅▄▇▄▅▃▆▆▇▆▇▆▇▇▅████▆█▇▆▇▇▇█
Accuracy,▁▂▃▃▄▆▄▄▃▅▄▆▆▆▆▇█▅██▇█▆█▇▆▇▇▇▇
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


Total number of parameters is: 163153
{'num_epochs': 30, 'lr': 0.0001, 'batch_size': 64, 'transforms': [Compose(
    RandomApply(
    p=0.6
    RandomVerticalFlip(p=0.2)
    RandomHorizontalFlip(p=0.9)
    RandomAffine(degrees=[-1.0, 1.0], translate=(0.1, 0.1), scale=(0.95, 0.95))
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 3))
)
    Normalize(mean=826.7405, std=2492.394)
), Compose(
    RandomApply(
    p=0.6
    RandomFlip_Voxel(p=0.8,dim=3)
    RandomFlip_Voxel(p=0.8,dim=1)
    RandomFlip_Voxel(p=0.2,dim=2)
)
    Normalize_Voxel(mean=12.3799,std=260.7471)
)], 'dropout': 0, 'channels': 8, 'heat_type': 'VGG', 'vox_type': 'VGG', 'num_params': 163153, 'batchnorm': True, 'datatype': 'heatmap_voxel_hull'}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 0 :::::: Train Loss: 0.707165002822876
              ____________ 
              :: Val Loss: 0.8683686852455139
              :: Val Accuracy: 0.4575645756457565
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 1 :::::: Train

train_loss,0.12967
val_loss,0.87238
Precision,0.9375
recall,0.6
F1,0.73171
Accuracy,0.75646
_runtime,831
_timestamp,1629321785
_step,29


train_loss,█▇▇▆▆▅▅▅▄▃▃▃▃▃▃▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁
val_loss,▆▅▄▄▄▃▄▁▂▁▄▁▅▇▅▆█▅▂█▅▃▅▃▁▄▅▂▅▆
Precision,█▁▁▁▂▂▂▅▅▄▅▃▄▄▅▅▆▄▅▇▆▆▆▃▆▆▆▆▄▇
recall,▁▆▆▆▆▆▆▆▆▇▆▇▅▅▄▅▅▆▇▄▅▆▅█▆▆▆▆▇▆
F1,▁▆▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▇█▆▆▇▇█▇▇▇███
Accuracy,▁▄▄▄▅▅▅▇▇▇▆▇▅▅▅▆▆▇▇▅▆▇▇█▇███▇█
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


Total number of parameters is: 163153
{'num_epochs': 30, 'lr': 0.0001, 'batch_size': 64, 'transforms': [Compose(
    RandomApply(
    p=0.6
    RandomVerticalFlip(p=0.2)
    RandomHorizontalFlip(p=0.9)
    RandomAffine(degrees=[-1.0, 1.0], translate=(0.1, 0.1), scale=(0.95, 0.95))
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 3))
)
    Normalize(mean=826.7405, std=2492.394)
), Compose(
    RandomApply(
    p=0.6
    RandomFlip_Voxel(p=0.8,dim=3)
    RandomFlip_Voxel(p=0.8,dim=1)
    RandomFlip_Voxel(p=0.2,dim=2)
)
    Normalize_Voxel(mean=12.3799,std=260.7471)
)], 'dropout': 0.1, 'channels': 8, 'heat_type': 'VGG', 'vox_type': 'VGG', 'num_params': 163153, 'batchnorm': True, 'datatype': 'heatmap_voxel_hull'}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 0 :::::: Train Loss: 0.7082959413528442
              ____________ 
              :: Val Loss: 0.8679159283638
              :: Val Accuracy: 0.4612546125461255
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Epoch 1 :::::: Train